In [3]:
!pip install impyla

     |████████████████████████████████| 245kB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 6.9MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 7.8MB/s eta 0:00:011
     |████████████████████████████████| 307kB 6.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 8.3MB/s eta 0:00:011
  Created wheel for impyla: filename=impyla-0.16.2-cp37-none-any.whl size=254680 sha256=f7b8d37c29f7589f920961a5bb5a8c656475266ad3aab228b7f56670dda4d1e0
  Stored in directory: /home/jovyan/.cache/pip/wheels/08/64/08/7804d27f94a792cffa3d8e2bcf57ba031c796b1b28b004662e
  Created wheel for bitarray: filename=bitarray-1.2.1-cp37-cp37m-linux_x86_64.whl size=165657 sha256=ed0d5d792332da84d9913c90b63db636a7142f37d50276b92ffb9c7a480ae235
  Stored in directory: /home/jovyan/.cache/pip/wheels/8e/4f/42/90d76aca9f4a7420c8c82fe15ed949777ab3fafe9fc1d705b3
  Created wheel for thrift: filename=thrift-0.13.0-cp37-cp37m-linux_x86_64.whl size=409398 sha256=19c6d5dc83f20f35eed5aae6b

In [21]:
!pip install ibis-framework 

     |████████████████████████████████| 542kB 3.9MB/s eta 0:00:01
     |████████████████████████████████| 696kB 5.9MB/s eta 0:00:01


In [27]:
!pip install ibis-framework[impala]

     |████████████████████████████████| 51kB 2.9MB/s eta 0:00:011
  Created wheel for hdfs: filename=hdfs-2.5.8-cp37-none-any.whl size=33213 sha256=8bd9865ccfdc8fe54474a459d644271d9e52a47dc07445a0fabaa01d601dee25
  Stored in directory: /home/jovyan/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=755bcad42dc1c624fa389d8da70f1d07ed6597fd5e647ed12ca3f0df005b4dd9
  Stored in directory: /home/jovyan/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built hdfs docopt


In [4]:
from impala.dbapi import connect

In [6]:
impala_server = 'ec2-54-252-190-231.ap-southeast-2.compute.amazonaws.com'

# connect to impala instance
conn = connect(host=impala_server, port=21050)
cursor = conn.cursor()

In [9]:
cursor.execute(""" CREATE TABLE Wind (rowid INT, Speed REAL, SpeedError REAL, Direction REAL)""")

In [ ]:
## generate data

In [10]:
import numpy as np
import math
import pandas as pd
import datetime as dt

In [11]:
windVal = []
windError = []
windOrientation = []
prevVal = 20
prevOrientation = np.random.uniform(0, 360)
for i in range(0, 86400):
    windVal.append(abs(np.random.normal(prevVal, 2, 1)[0]))
    windError.append(abs(np.random.normal(round(prevVal/10), 1)))
    if(i % 100 == 0):
        windOrientation.append(np.random.uniform(prevOrientation-50,
                                                 prevOrientation+50))
    else:
        windOrientation.append(np.random.uniform(prevOrientation-5,
                                                 prevOrientation+5))
    if(round(windVal[-1]) > 45):
        prevVal = int(math.floor(windVal[-1]))
    elif(round(windVal[-1]) < 10):
        prevVal = int(math.ceil(windVal[-1]))
    else:
        prevVal = int(round(windVal[-1]))
    prevOrientation = windOrientation[-1]

In [95]:
df = pd.DataFrame.from_dict({
                             'speed': windVal,
                             'speedError': windError,
                             'direction': windOrientation
                            })

In [13]:
now = dt.datetime.now()
sec = now.second
minute = now.minute
hour = now.hour

totalTime = (hour * 3600) + (minute * 60) + (sec)

In [15]:
df.Speed.size

86400

In [96]:
df['rowid'] = df.index

In [97]:
df

,speed,speedError,direction,rowid
0,19.891298,2.922832,67.075006,0
1,21.068451,0.331966,67.727730,1
2,20.654959,1.754218,71.556539,2
3,20.349572,1.109214,72.011781,3
4,21.043208,1.436588,76.555400,4
5,19.857643,4.057331,76.730515,5
6,19.834821,2.505773,79.180579,6
7,20.569028,1.325231,79.974621,7
8,23.116848,2.029249,78.339817,8
9,25.515597,3.246588,75.120138,9


In [82]:
# testing hdfs
from hdfs import InsecureClient
client = InsecureClient('http://' + impala_server + ':9870', user='admin')

In [83]:
content = client.content('/user/admin')

In [84]:
#df.to_sql(name='Wind', con=conn)
# ibis
import ibis

# need to fix hdfs
# port is NameNode Web UI Port
hdfs = ibis.hdfs_connect(host=impala_server, port=9870, protocol='webhdfs')
con = ibis.impala.connect(host=impala_server, database='default', hdfs_client=hdfs)

In [101]:
wind_table = con.table('wind', database='default') 

In [102]:
wind_table

ImpalaDatabaseTable[table]
  name: default.`wind`
  schema:
    rowid : int32
    speed : float64
    speederror : float64
    direction : float64

In [98]:
df.columns

Index(['speed', 'speedError', 'direction', 'rowid'], dtype='object')

In [104]:
df.rowid

0            0
1            1
2            2
3            3
4            4
         ...  
86395    86395
86396    86396
86397    86397
86398    86398
86399    86399
Name: rowid, Length: 86400, dtype: int64

In [100]:
wind_table.insert(df, overwrite=True)

OperationalError: Operation is in ERROR_STATE